# Useful links
pyxem
1. Documentation: https://pyxem.readthedocs.io/en/stable/user_guide/installing.html

py4dStem
1. Documentation: https://py4dstem.readthedocs.io/en/latest/api/py4DSTEM.html
2. https://github.com/py4dstem/py4DSTEM/blob/dev/py4DSTEM/process/strain/strain.py
3. 

other
1. https://www.nature.com/articles/s41524-022-00939-9#code-availability
2. CODE FROM: https://github.com/py4dstem/py4DSTEM_tutorials/blob/main/notebooks/strain_LFP.ipynb
3. https://github.com/AI-ML-4DSTEM/4D-OPTIMIZE/tree/nersc_ray

# Read file

What parameters are necessary? initial ROI, QR rotation, recip px size, real px size, kernel (C2 size, Camera Length), datacube, ...

In [ ]:
#Libraries
import pandas as pd
import py4DSTEM
from py4DSTEM import show
import numpy as np
import matplotlib.pyplot as plt
print('py4DSTEM version: {}'.format(py4DSTEM.__version__))

NameError: name 'filepath' is not defined

# Using py4dstem
Strain in 4DSTEM is single phase strain.
If you are trying to find the strain you are going to need to assume the user looking at a material that is all the same phase.

To find strain you will have a region identified as your baseline. This is your 0 strain region. Everywhere else is 0,+ or - in strain. 
All diffraction patterns will need to be compared to this region. 


Orientation indexing (with actual material indeces) will need to be compared to a .cif file. This is performed more widely and should* be simpler to implement.
Orientation indexing will need to begin by simulating the .cif file structures. For a multi-phase multi material object take a look at the pyxem documentation (I dont think py4dstem mentions this for multi-component multi-material systems)



In [ ]:
#Establish file path
filepath_data = r"c:\Users\a.santana\Downloads\Sb2S3_Prec1deg_Ort2_DPs.hspy"

In [ ]:
py4DSTEM.print_h5_tree( filepath_data )

In [ ]:
# Load data
datacube = py4DSTEM.read( filepath = filepath_data )

# print results
print(datacube)

In [ ]:
# Mean and max diffraction pattern of the calibration and measurement datasets

# compute
dp_mean = datacube.get_dp_mean()
dp_max = datacube.get_dp_max()

# show
show(
    [dp_mean,dp_max],
    cmap = 'inferno',
    title = ['mean DP','max DP']
)

In [ ]:
# Virtual imaging - position the detectors


# We'll generate a bright-field (BF) and an annular dark-field (ADF) virtual image
# for both the calibration and measurement datasets


# Set the center
center = (69,64)


# set the geometries
radius_bf = 20
geometry_bf = (center,radius_bf)
geometry_adf = (center,(radius_bf,10e3))

# display the detector positions
datacube.position_detector(
    data = dp_max,
    mode = 'circle',
    geometry = geometry_bf
)
datacube.position_detector(
    data = dp_max,
    mode = 'annulus',
    geometry = geometry_adf
)

In [ ]:
# Virtual imaging - compute the images


# compute
im_bf = datacube.get_virtual_image(
    mode = 'circle',
    geometry = geometry_bf,
    name = 'bright_field_cal',
)
im_adf = datacube.get_virtual_image(
    mode = 'annulus',
    geometry = geometry_adf,
    name = 'dark_field_cal',
)

# show
py4DSTEM.show(
    [im_bf,im_adf],
    figsize=(16,2),
    bordercolor = 'w',
    cmap='gray',
    title=['BF','ADF']
)

# Construct the probe template

In [ ]:
# Examine scan points for a vacuum probe

rx,ry = 7,11

py4DSTEM.visualize.show_selected_dp(
    datacube,
    im_adf,
    rx,
    ry
)

In [ ]:
# Select a vacuum region

xlims = 4,10
ylims = 8,14

ROI = np.zeros(datacube.rshape, dtype=bool)
ROI[xlims[0]:xlims[1], ylims[0]:ylims[1]] = True

show(
    im_adf,
    mask = ROI,
    mask_color = 'r',
    mask_alpha = 0.5
)

In [ ]:
probe = datacube.get_vacuum_probe(
    ROI = ROI,
)

show(
    probe.probe,
    vmin=0,
    vmax=1,
)

In [ ]:
# Estimate the radius of the BF disk

probe_semiangle, probe_qx0, probe_qy0 = py4DSTEM.process.calibration.get_probe_size(probe.probe)

show(
    probe.probe,
    circle = {
        'center' : (probe_qx0,probe_qy0),
        'R' : probe_semiangle,
    },
    vmin = 0,
    vmax = 1
)

In [ ]:
probe.get_kernel(
    mode = 'sigmoid',
    origin = (probe_qx0,probe_qy0),
    radii = (probe_semiangle * 1,
             probe_semiangle * 4
    )
)

py4DSTEM.visualize.show_kernel(
    probe.kernel,
    R = 30,
    L = 30,
    W = 1
)

# Find Bragg Peaks

In [ ]:
# Choose some diffraction patterns to use for hyperparameter tuning


rxs = 45, 25, 25, 35, 25, 66
rys = 17, 39, 14, 17, 42, 25


colors=['deeppink','coral','gold','chartreuse','dodgerblue','rebeccapurple']


py4DSTEM.visualize.show_points(
    im_adf,
    x=rxs,
    y=rys,
    pointcolor=colors,
    figsize=(4,4)
)

py4DSTEM.visualize.show_image_grid(
    get_ar = lambda i:datacube.data[rxs[i],rys[i],:,:],
    H=2, 
    W=3,
    axsize=(5,5),
    scaling='log',
    vmin=0,
    vmax=1,
    get_bordercolor = lambda i:colors[i],
)

In [ ]:
# Tune disk detection parameters


# disk detection parameters
detect_params = {
    'corrPower': 1.0,
    'sigma': 0,
    'edgeBoundary': 4,
    'minRelativeIntensity': 0,
    'minAbsoluteIntensity': 8,
    'minPeakSpacing': 4,
    'subpixel': 'poly',
    # 'subpixel' : 'multicorr',
    'upsample_factor': 8,
    'maxNumPeaks': 1000,
#     'CUDA': True,
}
# Note that "poly" subpixel fitting can be used to keep this tutoral fast, but "multicorr"
# is more accurate. For high precision strain mapping, subpixel="multicorr" is recommended.


# find the selected disks
disks_selected = datacube.find_Bragg_disks(
    data = (rxs, rys),
    template = probe.kernel,
    **detect_params,
)

# show
py4DSTEM.visualize.show_image_grid(
    get_ar = lambda i:datacube.data[rxs[i],rys[i],:,:],
    H=2, 
    W=3,
    axsize=(5,5),
    scaling='log',
    vmin=0,
    vmax=1,
    get_bordercolor = lambda i:colors[i],
    get_x = lambda i: disks_selected[i].data['qx'],
    get_y = lambda i: disks_selected[i].data['qy'],
    get_pointcolors = lambda i: colors[i],
    open_circles = True,
    scale = 700,
)

In [ ]:
# Find Bragg peaks in all probe positions.


braggpeaks = datacube.find_Bragg_disks(
    template = probe.kernel,
    **detect_params,
)

# Calibration

## Center Coordinate System

In [ ]:
# Compute a Bragg vector map,
# i.e. a 2D histogram of the Bragg peak positions,
# weighted by their correlation intensities

bvm_raw = braggpeaks.histogram(mode = 'raw')

# Plot the BVM
bvm_vis_params = {
    'scaling':'power',
    'power':0.5,
    'intensity_range':'absolute',
    'vmin':0,
    'vmax':2e3
}
py4DSTEM.show(
    bvm_raw,
    **bvm_vis_params,
    returnfig = True
)

center guess can be done by finding the middle of a 256,256 image
or
by finding the most intense set of data till it drops off and masking it

In [ ]:
# Initial guess for the center position

center_guess = 128,128

py4DSTEM.show(
    bvm_raw,
    points = {'x':center_guess[0],'y':center_guess[1]},
    **bvm_vis_params,
)

In [ ]:
# Find the origin at each beam position

# measure the origin
origin_meas = braggpeaks.measure_origin( center_guess=center_guess )

# fit a 2D plane
qx0_fit,qy0_fit,qx0_residuals,qy0_residuals = braggpeaks.fit_origin(
    ticks = False,
    axsize = (4,4),
)

In [ ]:
# Get the Bragg vector map after centering using our new origin measurements

# compute
bvm_centered = braggpeaks.histogram()

# show
py4DSTEM.show(
    bvm_centered,
    **bvm_vis_params,
)
py4DSTEM.show(
    bvm_centered,
    vmin=0,
    vmax=1,
    circle = {
        'center' : bvm_centered.origin,
        'R' : 2,
    },
)

In [ ]:
# Get the Bragg vector map after centering using our new origin measurements

# compute
bvm_centered_masked = braggpeaks.histogram()

# show
py4DSTEM.show(
    bvm_centered_masked,
    **bvm_vis_params,
)
py4DSTEM.show(
    bvm_centered_masked,
    vmin=0,
    vmax=1,
    circle = {
        'center' : bvm_centered_masked.origin,
        'R' : 2,
    },
)

### Ellipticity

In [ ]:
# Select a region of thicker lacy carbon

xlims = 75,86
ylims = 35,43

ROI_amorph = np.zeros(datacube.rshape, dtype=bool)
ROI_amorph[xlims[0]:xlims[1], ylims[0]:ylims[1]] = True

show(
    im_adf,
    mask = ROI_amorph,
    mask_color = 'r',
    mask_alpha = 0.5
)

In [ ]:
im_SAED_amorph = datacube.get_virtual_diffraction(
    'mean',
    mask = ROI_amorph,
    shift_center = True
)

show(
    im_SAED_amorph,
    intensity_range = 'absolute',
    vmin = 4e1,
    vmax = 8e3,
    scaling = 'log'
)

In [ ]:
# Select fit radii

q_range = (16,36)

py4DSTEM.show(
    im_SAED_amorph.data,
    intensity_range = 'absolute',
    vmin = 4e1,
    vmax = 8e3,
    scaling = 'log',
    annulus={
        'center':im_SAED_amorph.calibration.get_origin_mean(),
        'radii': q_range,'fill':True,'color':'r','alpha':0.3}
)

In [ ]:
# Fit the amorphous halo

p_ellipse,p_dsg = py4DSTEM.process.calibration.fit_ellipse_amorphous_ring(
    data = im_SAED_amorph.data,
    center = im_SAED_amorph.calibration.get_origin_mean(),
    fitradii = q_range
)

# Show the fit
py4DSTEM.visualize.show_amorphous_ring_fit(
    im_SAED_amorph.data,
    fitradii = q_range,
    p_dsg = p_dsg,
)

In [ ]:
print(p_ellipse)

In [ ]:
# Set the ellipse calibration

braggpeaks.calibration.set_p_ellipse(p_ellipse)
braggpeaks.setcal()

In [ ]:
braggpeaks.calibration

In [ ]:
# Get a new corrected BVM

# compute
bvm_ellipse = braggpeaks.histogram( mode='cal' )


# show
show(
    [bvm_centered_masked, bvm_ellipse],
    vmax=0.99,
    title = ["bvm before ellipse fit", "bvm after ellipse fit"]
)

### Rotation

Measure QR Rotation with pseudo-dpc with only the center beam (QR rotation might be given by the emd or mrc file and this can then be ignored)

In [ ]:
# make a mask

mask_radius = 14

qyy,qxx = np.meshgrid(
    np.arange(datacube.qshape[1]),
    np.arange(datacube.qshape[0]),
)
origin = braggpeaks.calibration.get_origin_mean()
qq = np.hypot(qxx-origin[0],qyy-origin[1])
mask_pseudoDPC = qq<mask_radius

show(
    dp_max,
    vmax = 0.975,
    circle = {
        'center' : braggpeaks.calibration.get_origin_mean(),
        'R' : mask_radius
    }
)
show(
    dp_max,
    vmax = 0.975,
    mask = mask_pseudoDPC,
    mask_color = 'r',
    mask_alpha = 0.5
)

In [ ]:
# perform DPC preprocessing

dpc = py4DSTEM.process.phase.DPCReconstruction(
    datacube = datacube,
).preprocess(
    dp_mask = mask_pseudoDPC,
)

In [ ]:
# The rotation was measured at roughly -71 degrees and is input below manually.
# Can you find a way to confirm that this is the right rotation using only what you've seen so far in this notebook?

braggpeaks.calibration.set_QR_rotation_degrees(-71)
braggpeaks.calibration.set_QR_flip(True)
#braggpeaks.calibration.set_QR_rotation_degrees(-71)
#braggpeaks.calibration.set_QR_flip(False)
braggpeaks.setcal()

In [ ]:
# Update BVM

# compute
bvm = braggpeaks.histogram( mode='cal' )


# show
show([bvm_raw, bvm],vmax=0.99)

## Strain

In [ ]:
strainmap = py4DSTEM.StrainMap( braggvectors=braggpeaks )

In [ ]:
strainmap.choose_basis_vectors(
    minSpacing=10,
    minAbsoluteIntensity=1e3,
    maxNumPeaks=100,
    edgeBoundary=5,
    vis_params = bvm_vis_params,
    # index_g1 = 11,
    # index_g2 = 9,
)

In [ ]:
strainmap.fit_basis_vectors(
    max_peak_spacing = 3,
)

In [ ]:
# strain map

strainmap.get_strain(
#    coordinate_rotation = 0,
    coordinate_rotation = -55,
#    coordinate_rotation = -100,
    layout = "horizontal",
    vrange = [-6.0, 6.0],
    vrange_theta = [-2.0, 2.0],
    # mask_color = "blue"
)

In [ ]:
strainmap.show_strain(
    layout = "horizontal",
    vrange = [-6.0, 6.0],
    vrange_theta = [-2.0, 2.0],
    show_gvects = True,
)

In [ ]:
strainmap.show_reference_directions(
    visp_cal={
        'scaling' : 'none',
        'vmax' : 0.995
    },
    visp_uncal={
        'scaling' : 'none',
        'vmax' : 0.995
    },
    camera_length = 1.4 #This is inputted manually
)

## Strain using reference g1 and g2 and from an ROI

In [ ]:
# Set reference region

ROI = np.zeros(braggpeaks.Rshape, dtype=bool)
ROI[40:50, 20:25] = True 
# ^You need to determiine your ROI on a user by user basis... maybe unless in this notebook they are able to determine where
# there is 0 strain. Which i dont think they do.

show(
    im_adf,
    mask = ROI,
    mask_color='r',
    mask_alpha=0.4
)

What are these coordinate rotation values they use from? What are the vrange and vrange theta from?\

Seems to be coming from their process of rotating the sample. (QR is from the beam rotation)

vrange is related to the colorbar intensity shift. Not important to collecting data.

In [ ]:
# strain from a region

strainmap.get_strain(
    gvects = ROI,
    coordinate_rotation = 0,
    #coordinate_rotation = -55,
#    coordinate_rotation = -100,
    layout = "square",
    vrange = [-6.0, 6.0],
    vrange_theta = [-2.0, 2.0],
)

## Strain using a manually specified reference g1 and g2

In [ ]:
g1_ref, g2_ref = strainmap.get_reference_g1g2( ROI )

strainmap.get_strain(
    gvects = (g1_ref,g2_ref),
    coordinate_rotation = 0,
    layout = "horizontal",
    vrange = [-6.0, 6.0],
    vrange_theta = [-2.0, 2.0],
)

In [ ]:
# end
#https://github.com/py4dstem/py4DSTEM_tutorials/blob/main/notebooks/strain_LFP.ipynb